#### LICENSE
These notes are released under the 
"Creative Commons Attribution-ShareAlike 4.0 International" license. 
See the **human-readable version** [here](https://creativecommons.org/licenses/by-sa/4.0/)
and the **real thing** [here](https://creativecommons.org/licenses/by-sa/4.0/legalcode). 

#### INSTALLATION instructions

To use this noteboook you may need to install a few packages in `R`:
```
install.packages(c('rmutil', 'robustbase', 'RobStatTM'))
```

# Intro to M-estimators

Here we will compute simple M-estimators. 

Start with a simple sample. 

In [ ]:
set.seed(123)
n <- 30
x <- rnorm(30, mean=5, sd=1.5)
rbind(mean = mean(x), median = median(x))

We now compute an M-estimator, using a Huber loss, and without standardizing. We 
write our own code. 

In [ ]:
huberPsi <- function(r, cc)
    return( pmin(pmax(-cc, r), cc) )

In [ ]:
mest0 <- function(x, cc=1.345, init=median(x), max.it = 100, eps=1e-8) {
    m1 <- init
    m0 <- m1 + 10*eps
    it <- 0
    while( ((it <- it+1) < max.it ) & (abs(m1-m0) > eps ) ) {
        re <- (x - m1)
        w <- huberPsi(re, cc=cc)/re
        w[ is.na(w) ] <- 1
        m0 <- m1
        m1 <- sum( x*w ) / sum(w)
    }
    return(m1)
}

Now, let's compute the M-estimator

In [ ]:
(mest0(x))

What if we divide all the data by 100?

In [ ]:
c(mean(x), mean(x/100)*100)
c(median(x), median(x/100)*100)
c(mest0(x), mest0(x/100)*100)

Let us add 5 outliers, and see what happens.

In [ ]:
xc <- c(x, rnorm(5, mean=-20, sd=.5))
c(mean(x), mean(xc))
c(median(x), median(xc))
c(mest0(x), mest0(xc))

What if we now again divide all the data by 100? 

In [ ]:
c(mean(xc), mean(xc/100)*100)
c(median(xc), median(xc/100)*100)
c(mest0(xc), mest0(xc/100)*100)

The M estimator is equal to the sample mean. Uninformative!

We need to standardize!

In [ ]:
mest <- function(x, cc=1.345, init=median(x), si = mad(x), max.it = 100, eps=1e-8) {
    m1 <- init
    m0 <- m1 + 10*eps
    it <- 0
    while( ((it <- it+1) < max.it ) & (abs(m1-m0) > eps ) ) {
        re <- (x - m1) / si
        w <- huberPsi(re, cc=cc)/re
        w[ is.na(w) ] <- 1
        m0 <- m1
        m1 <- sum( x*w ) / sum(w)
    }
    return(m1)
}

And now everything works fine!

In [ ]:
c(mean(xc), mean(xc/100)*100)
c(median(xc), median(xc/100)*100)
c(mest(xc), mest(xc/100)*100)